In [2]:
import sys

from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from minio import Minio
from utils.common import get_spark_session, get_minio_client
from utils.bronze import Bronze

username = 'kayden'

spark = get_spark_session(
        f"{username}_ETL",
        username,
    )

log_path = f"s3a://{username}/logging"

spark._jsc.hadoopConfiguration().set("log4j.appender.file", "org.apache.log4j.FileAppender")
spark._jsc.hadoopConfiguration().set("log4j.appender.file.File", log_path)
spark._jsc.hadoopConfiguration().set("log4j.appender.file.layout", "org.apache.log4j.PatternLayout")
spark._jsc.hadoopConfiguration().set("log4j.appender.file.layout.ConversionPattern", "%d{yyyy-MM-dd HH:mm:ss} %-5p %c{1}:%L - %m%n")

    # mc = get_minio_client(username)
MINIO_ENDPOINT = "minio:9000/"
MINIO_ACCESS_KEY = username
MINIO_SECRET_KEY = "password"

mc = Minio(
        endpoint=MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
    )

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3d2ec19e-867d-4850-80fa-c3c6935f7847;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1531ms)
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (470ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-st

In [3]:
spark.sql("SHOW DATABASES")

24/05/31 07:08:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[namespace: string]

In [5]:
spark.sql("CREATE DATABASE bronze_kayden")
spark.sql("use bronze_kayden")
spark.sql("create table shopee_sales using delta location 's3a://kayden/bronze/shopee_sales'")

24/05/30 21:04:26 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`bronze_kayden`.`shopee_sales` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/05/30 21:04:26 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [6]:
result_df = spark.sql("""
    SELECT
        QUARTER(order_date) AS quarter,
        DATE_FORMAT(order_date, 'EEEE') AS day_of_week,
        DAYOFWEEK(order_date) AS day_of_week_number
    FROM
        bronze_kayden.shopee_sales
""")

# Show the result DataFrame
result_df.show()

24/05/30 21:09:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------+------------------+
|quarter|day_of_week|day_of_week_number|
+-------+-----------+------------------+
|      2|     Monday|                 2|
|      2|    Tuesday|                 3|
|      2|  Wednesday|                 4|
|      2|   Thursday|                 5|
|      2|     Friday|                 6|
+-------+-----------+------------------+



In [7]:
spark.sql(f"USE bronze_{username}")
table = f"bronze_{username}.shopee_sales"
query = f"""
    SELECT 
        order_number,
        order_date,
        ship_date,
        ordered_quantity as order_quantity,
        unit_price,
        unit_discount,
        CAST(total_order_price AS FLOAT) as sales_amount,
        payment_date,
        product_key,
        product_name,
        category_name as product_category,
        unit_price as price,
        weight,
        DAY(order_date) AS day,
        MONTH(order_date) AS month,
        YEAR(order_date) as year,
        QUARTER(order_date) as quarter,
        DATE_FORMAT(order_date, 'EEEE') AS day_of_week,
        DAYOFWEEK(order_date) AS day_of_week_number,
        shop_discount_code,
        shopee_discount_code,
        tracking_code,
        shipping_company,
        buyer_name as customer_name
    FROM {table}
"""

sql_transformed_df = spark.sql(query)

sql_transformed_df.printSchema()

root
 |-- order_number: string (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- ship_date: string (nullable = true)
 |-- order_quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- unit_discount: double (nullable = true)
 |-- sales_amount: float (nullable = true)
 |-- payment_date: timestamp (nullable = true)
 |-- product_key: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_number: integer (nullable = true)
 |-- shop_discount_code: string (nullable = true)
 |-- shopee_discount_code: string (nullable = true)
 |-- tracking_code: string (nullable = true)
 |-- shipping_company: string (nullable

In [14]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
# sql_transformed_df.toPandas().head(5)
sql_transformed_df.show(10)

+--------------------+-------------------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|        order_number|         order_date| ship_date|order_quantity|unit_price|unit_discount|sales_amount|payment_date|product_key|        product_name|product_category|   price|weight|day|month|year|quarter|day_of_week|day_of_week_number|shop_discount_code|shopee_discount_code|tracking_code|shipping_company|       customer_name|
+--------------------+-------------------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|SO2

In [16]:
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import col, when, last, desc, dense_rank, to_date, date_format
cleaned_df = sql_transformed_df \
            .filter(col("order_number").isNotNull() & col("product_key").isNotNull() & col("order_date").isNotNull()) \
            .withColumn("product_name", when(col("product_name").isNull(), 
                                            last("product_name", ignorenulls=True)
                                            .over(Window.partitionBy("product_key").orderBy("order_date")))
                        .otherwise(col("product_name"))) \
            .na.drop(subset=["order_number", "product_key", "order_date", "product_name"]) \
            .withColumn("rank", dense_rank().over(Window.partitionBy("order_number", "product_key").orderBy(desc("order_date")))) \
            .filter(col("rank") == 1) \
            .drop("rank")
cleaned_df.show(10)
null_core_columns = cleaned_df.filter(col("order_number").isNull() | col("product_key").isNull() | col("order_date").isNull() | col("product_name").isNull())
    
    # Count the number of rows with null core columns
null_core_count = null_core_columns.count()

+--------------------+-------------------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|        order_number|         order_date| ship_date|order_quantity|unit_price|unit_discount|sales_amount|payment_date|product_key|        product_name|product_category|   price|weight|day|month|year|quarter|day_of_week|day_of_week_number|shop_discount_code|shopee_discount_code|tracking_code|shipping_company|       customer_name|
+--------------------+-------------------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|SO2

In [17]:
print("number of rows with null core columns:", null_core_count)

number of rows with null core columns: 0


In [19]:
cleaned_df.printSchema()

root
 |-- order_number: string (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- ship_date: string (nullable = true)
 |-- order_quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- unit_discount: double (nullable = true)
 |-- sales_amount: float (nullable = true)
 |-- payment_date: timestamp (nullable = true)
 |-- product_key: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_number: integer (nullable = true)
 |-- shop_discount_code: string (nullable = true)
 |-- shopee_discount_code: string (nullable = true)
 |-- tracking_code: string (nullable = true)
 |-- shipping_company: string (nullable

In [20]:
numeric_columns = ['unit_price', 'unit_discount', 'sales_amount', 'price', 'weight']
for col_name in numeric_columns:
    cleaned_df = cleaned_df.withColumn(col_name, when(col(col_name).isNull() | (col(col_name) < 0), 0).otherwise(col(col_name)))

cleaned_df = cleaned_df.withColumn('order_quantity', when(col('order_quantity').isNull() | (col('order_quantity') == 0), 1).otherwise(col('order_quantity')))

# Replace null or 0 values in sales_amount with (unit_price * ordered_quantity) - unit_discount
cleaned_df = cleaned_df.withColumn('sales_amount', when((col('sales_amount').isNull()) | (col('sales_amount') == 0), (col('unit_price') * col('order_quantity')) - col('unit_discount')).otherwise(col('sales_amount')))
    
# Format date columns to 'yyyy-MM-dd'
date_columns = ['order_date', 'ship_date', 'payment_date']
for col_name in date_columns:
    cleaned_df = cleaned_df.withColumn(col_name, to_date(col(col_name), 'yyyy-MM-dd').cast('date'))

# Format order_date to 'yyyy-MM-dd'
cleaned_df = cleaned_df.withColumn('order_date', date_format(col('order_date'), 'yyyy-MM-dd'))

cleaned_df.show()

+--------------------+----------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|        order_number|order_date| ship_date|order_quantity|unit_price|unit_discount|sales_amount|payment_date|product_key|        product_name|product_category|   price|weight|day|month|year|quarter|day_of_week|day_of_week_number|shop_discount_code|shopee_discount_code|tracking_code|shipping_company|       customer_name|
+--------------------+----------+----------+--------------+----------+-------------+------------+------------+-----------+--------------------+----------------+--------+------+---+-----+----+-------+-----------+------------------+------------------+--------------------+-------------+----------------+--------------------+
|SO20210101/SHOPPE-10|2021-01-0